# MPS Simplification

## Algorithm

The "simplification" algorithm consists on approximating a quantum state $|\Psi\rangle,$ encoded in a large bond dimension MPS, using an MPS with a smaller bond dimension fixed bond dimension $\xi$, $|\phi\rangle\in\mathrm{MPS}_\xi$. We work by minimizing the distance between both states
$$\phi = \mathrm{argmin}_{\phi\in \mathrm{MPS}_\xi} d(\Psi,\phi).$$

The algorithm uses this distance expressed as a sum of scalar products
$$d(\Psi,\phi) = \|\Psi-\phi\|^2 = \langle\Psi|\Psi\rangle + \langle\phi|\phi\rangle - \langle\Psi|\phi\rangle - \langle\phi|\Psi\rangle$$

Given the structure of scalar products we see that the distance is a bilinear function with respect to any of the tensors in $|\phi\rangle.$ Take for instance an MPS with four sites and let us focus on the second site. The scalar product between $\Psi$ and $\phi$ can be written as the contraction between two environments and two tensors from each state

<img src="figures/scalar-product-with-environments.svg" style="max-width:90%; width:30em">

As explained in [this notebook](File%201c%20-%20Canonical%20form.ipynb), if $|\phi\rangle$ is in canonical form with respect to that site., its norm can be written directly as the contraction of the same tensor and its complex conjugate.

<img src="figures/scalar-product-canonical.svg" style="max-width:90%; width:30em">

This structure allows us to write the condition for optimality with respect to a site
$$\frac{\partial}{\partial D^{i}_{\alpha\beta}} d(\Psi,\phi) = 0$$
as a linear equation with respect to the tensor
$$U^{i}_{\alpha\beta} - D^{i}_{\alpha\beta} = 0$$
with the elements
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\Psi|\phi\rangle = U^{i}_{\alpha\beta}$$
and
$$\frac{\partial}{\partial D^{i *}_{\alpha\beta}} \langle\phi|\phi\rangle = D^{i}_{\alpha\beta}$$

The algorithm in this form has a drawback: *we need to know the size of the tensors* in advance. An useful variant is to optimize with respect to two sites, for instance $D$ and $F$ simultaneously, combining them in a single tensor that satisfies a similar equation. The two-site tensor is then split optimally, distributing the entanglement according to the maximum bond dimension that is allowed.

<img src="figures/two-site-optimization.svg" style="max-width:70%; width: 70em">

## Implementation

The code is very simple, but it becomes more so if we abstract out the environment part into a single object, a linear form
$$L(|\phi\rangle) := \langle\psi|\phi\rangle,$$
and we instruct this object to return the local terms $L$, $C$ and $R$ with respect to the current site.

In [ ]:
# file: mps/truncate.py

import numpy as np
from mps.state import *

from mps.expectation import \
    begin_environment, \
    update_right_environment, \
    update_left_environment

class LinearForm(object):
    #
    # This class is an object that formally implements <ψ|ϕ> with an argument
    # ϕ that may change and be updated over time.
    #
    # Given a site 'n' it returns the tensor 'L' such that the contraction
    # between 'L' and ϕ[n] is the result of the linear form."""
    #
    def __init__(self, ψ, ϕ, center=0):
        #
        # At the beginning, we create the right- and left- environments for
        # all sites to the left and to the right of 'center', which is the
        # focus point of 'LinearForm'.
        #
        ρ = begin_environment()
        self.R = [ρ] * ψ.size
        for i in range(ψ.size-2, center-1, -1):
            self.R[i] = ρ = update_right_environment(ψ[i+1], ϕ[i+1], ρ)

        ρ = begin_environment()
        self.L = [ρ] * ψ.size        
        for i in range(1, center+1):
            self.L[i] = ρ = update_left_environment(ψ[i-1], ϕ[i-1], ρ)

        self.ψ = ψ
        self.ϕ = ϕ
        self.center = center

    def tensor(self):
        #
        # Return the tensor that represents the LinearForm at the 'center'
        # site of the MPS
        #
        center = self.center
        L = self.L[center]
        R = self.R[center]
        C = self.ψ[center].conj()
        
        return np.einsum('nk,njl,il->kji', L, C, R)

    def update(self, direction):
        #
        # We have updated 'ϕ', which is now centered on a different point.
        # We have to recompute the environments.
        #
        center = self.center
        if direction > 0:
            L = self.L[center]
            if center+1 < ψ.size:
                L = update_left_environment(ψ[center], ϕ[center], L)
                self.L[center+1] = L
                self.center = center+1
        else:
            R = self.R[center]
            if center > 0:
                R = update_right_environment(ψ[center], ϕ[center], L)
                self.R[center-1] = R
                self.center = center-1

In [ ]:
# file: mps/truncate.py

def simplify_mps_1site(ψ, dimension=0, sweeps=2, tolerance=DEFAULT_TOLERANCE):
    """Simplify an MPS ψ transforming it into another one with a smaller bond
    dimension, sweeping until convergence is achieved.
    
    Arguments:
    ----------
    sweeps = maximum number of sweeps to run
    tolerance = relative tolerance when splitting the tensors
    dimension = maximum bond dimension, 0 to just truncate to tolerance
    """
    
    if dimension == 0:
        return CanonicalMPS(ψ, center=0, tolerance=tolerance)

---

# Tests

In [ ]:
# file: mps/test/test_truncate.py

import unittest
from mps.test.tools import *

class TestLinearForm(unittest.TestCase):
    
    def test_canonical_env(self):
        #
        # When we pass two identical canonical form MPS to LinearForm, the
        # left and right environments are the identity
        #
        def ok(ψ):
            for center in range(ψ.size):
                ϕ = CanonicalMPS(ψ, center)
                LF = LinearForm(ϕ, ϕ, center)
                self.assertTrue(almostIdentity(LF.L[center],+1))
                self.assertTrue(almostIdentity(LF.R[center],+1))
        
        test_over_random_mps(ok)
    
    def test_tensor(self):
        #
        # Use ψ and ϕ=O*ψ, where O is a local operator, to verify that the
        # LinearForm of ψ and ϕ returns <ψ|O|ψ>
        #
        def ok(ψ):
            O = np.array([[1,-3],[2,-4]])
            for i in range(ψ.size):
                Omean = ψ.expectation1(O, i)
                ϕ = ψ.copy()
                ϕ[i] = np.einsum('ij,ajb->aib', O, ϕ[i])
                for center in range(ϕ.size):
                    LF = LinearForm(ψ, ϕ, center)
                    L = LF.tensor()
                    A = ϕ[center]
                    Odesired = np.einsum('aib,aib', L, A)
                    print((Omean, Odesired))
                    self.assertTrue(almostIdentity(LF.L[center],+1))
                    self.assertTrue(almostIdentity(LF.R[center],+1))
        
        test_over_random_mps(ok)

In [ ]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestLinearForm'])
unittest.TextTestRunner(verbosity=2).run(suite1);

In [ ]:
mps.test.tools.__dict__.keys()